In [5]:
import os
import logging
import sys
import struct
from array import array
import random
import numpy as np
from models.weight_transfer_model2 import *
from utils import classification_batch_evaluation, hist_loss_batch_eval, proto_episodic_performance, proto_performance


import tensorflow as tf
import os
from tensorflow.python import keras

ModuleNotFoundError: No module named 'keras'

In [ ]:
##Boilerplate for loading Data

In [ ]:
class MNIST():
    def __init__(self, path):
        self.path = path

        self.test_img_fname = 't10k-images-idx3-ubyte'
        self.test_lbl_fname = 't10k-labels-idx1-ubyte'

        self.train_img_fname = 'train-images-idx3-ubyte'
        self.train_lbl_fname = 'train-labels-idx1-ubyte'

        self.test_images = []
        self.test_labels = []

        self.train_images = []
        self.train_labels = []

        self.num_classes = 10

    def load_testing(self):
        ims, labels = self.load(os.path.join(self.path, self.test_img_fname),
                                os.path.join(self.path, self.test_lbl_fname))

        self.test_images = self.process_images(ims)
        self.test_labels = self.process_labels(labels)

        return self.test_images, self.test_labels

    def load_training(self):
        ims, labels = self.load(os.path.join(self.path, self.train_img_fname),
                                os.path.join(self.path, self.train_lbl_fname))

        self.train_images = self.process_images(ims)
        self.train_labels = self.process_labels(labels)

        return self.train_images, self.train_labels

    def process_images(self, images):
        images_np = np.array(images) / 255.0
        return images_np

    def process_labels(self, labels):
        return np.array(labels)

    @classmethod
    def load(cls, path_img, path_lbl):
        with open(path_lbl, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049,'
                                 'got {}'.format(magic))

            labels = array("B", file.read())

        with open(path_img, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051,'
                                 'got {}'.format(magic))

            image_data = array("B", file.read())

        images = []
        for i in range(size):
            images.append([0] * rows * cols)

        for i in range(size):
            images[i][:] = image_data[i * rows * cols:(i + 1) * rows * cols]

        return images, labels

    def kntl_data_form(self, t1_train, t1_valid, k, n, t2_test):
        self.load_testing()
        self.load_training()

        self.x_train_lt5_full = self.train_images[self.train_labels < 5]
        self.y_train_lt5_full = self.train_labels[self.train_labels < 5]
        shuffle = np.random.permutation(len(self.y_train_lt5_full))
        self.x_train_lt5_full, self.y_train_lt5_full = self.x_train_lt5_full[shuffle], self.y_train_lt5_full[shuffle]

        x_test_lt5 = self.test_images[self.test_labels < 5]
        y_test_lt5 = self.test_labels[self.test_labels < 5]
        shuffle = np.random.permutation(len(y_test_lt5))
        self.x_test_lt5_full, self.y_test_lt5_full = x_test_lt5[shuffle], y_test_lt5[shuffle]

        self.x_lt5 = np.concatenate((self.x_train_lt5_full, self.x_test_lt5_full), axis=0)
        self.y_lt5 = np.concatenate((self.y_train_lt5_full, self.y_test_lt5_full), axis=0)

        print('Task 1 full: {0}'.format(len(self.x_lt5)))

        shuffle = np.random.permutation(len(self.y_lt5))
        self.x_lt5, self.y_lt5 = self.x_lt5[shuffle], self.y_lt5[shuffle]
        self.x_valid_lt5, self.y_valid_lt5 = self.x_lt5[:t1_valid], self.y_lt5[:t1_valid]
        self.x_train_lt5, self.y_train_lt5 = self.x_lt5[t1_valid:t1_valid + t1_train], self.y_lt5[t1_valid:t1_valid + t1_train]

        print('Task 1 training: {0}'.format(len(self.x_train_lt5)))
        print('Task 1 validation: {0}'.format(len(self.x_valid_lt5)))

        self.x_train_gte5_full = self.train_images[self.train_labels >= 5]
        self.y_train_gte5_full = self.train_labels[self.train_labels >= 5] - 5
        self.x_test_gte5_full = self.test_images[self.test_labels >= 5]
        self.y_test_gte5_full = self.test_labels[self.test_labels >= 5] - 5

        self.x_gte5 = np.concatenate((self.x_train_gte5_full, self.x_test_gte5_full), axis=0)
        self.y_gte5 = np.concatenate((self.y_train_gte5_full, self.y_test_gte5_full), axis=0)

        print('Task 2 full: {0}'.format(len(self.x_gte5)))

        self.x_train_gte5 = []
        self.y_train_gte5 = []
        classes = np.unique(self.y_gte5)
        chosen_classes = np.random.choice(classes, n, replace=False)
        for c in chosen_classes:
            idx = np.random.choice(np.where(self.y_gte5 == c)[0], k, replace=False)
            self.x_train_gte5.extend(self.x_gte5[idx])
            self.y_train_gte5.extend(self.y_gte5[idx])
            self.x_gte5 = np.delete(self.x_gte5, idx, axis=0)
            self.y_gte5 = np.delete(self.y_gte5, idx, axis=0)

        self.x_train_gte5 = np.array(self.x_train_gte5)
        self.y_train_gte5 = np.array(self.y_train_gte5)

        assert t2_test <= len(self.y_gte5)

        shuffle = np.random.permutation(len(self.y_gte5))
        self.x_gte5, self.y_gte5 = self.x_gte5[shuffle], self.y_gte5[shuffle]
        self.x_test_gte5, self.y_test_gte5 = self.x_gte5[:t2_test], self.y_gte5[:t2_test]

        print('k = {0}, n = {1}'.format(k, n))
        print('Task 2 training: {0}'.format(len(self.x_train_gte5)))
        print('Task 2 test: {0}\n'.format(len(self.x_test_gte5)))

        return (self.x_train_lt5, self.y_train_lt5), (self.x_valid_lt5, self.y_valid_lt5), (self.x_train_gte5, self.y_train_gte5), (self.x_test_gte5, self.y_test_gte5)

In [2]:
params={'dataset': 'mnist', 'data_path': '/home/abhishek/Desktop/ANU/comp_6470/adapted_deep_embeddings/datasets/mnist', 't1_train': 8000, 't1_valid': 3000, 'k': 10, 'n': 5, 't2_test': 10000, 'epochs': 500, 'batch_size': 2048, 'learning_rate': 0.005, 'patience': 20, 'percentage_es': 0.01, 'random_seed': 1234, 'replications': 1, 'gpu': '0', 'controller': '/cpu:0', 'save_dir': '/home/abhishek/Desktop/ANU/comp_6470/adapted_deep_embeddings/trained_models/mnist/mnist_10_5/weight_transfer', 'log_file': '/home/abhishek/Desktop/ANU/comp_6470/adapted_deep_embeddings/trained_models/mnist/mnist_10_5/weight_transfer/log.txt', 'command': 'weight_transfer'}

In [3]:
data = MNIST(params['data_path']).kntl_data_form(params['t1_train'], params['t1_valid'],params['k'], params['n'], params['t2_test'])

NameError: name 'MNIST' is not defined

In [ ]:

def train_classification(sess, model, data, params, weight_transfer=True):
    (x_train, y_train), (x_valid, y_valid), (x_train2, y_train2), (x_test2, y_test2) = data

    if weight_transfer:
        initial_best_epoch = {'epoch': -1, 'valid_acc': -1}

        for epoch in range(1, params['epochs'] + 1):
#         for epoch in range(1, 10):

            shuffle = np.random.permutation(len(y_train))
            x_train, y_train = x_train[shuffle], y_train[shuffle]
            for i in range(0, len(y_train), params['batch_size']):
                x_train_mb, y_train_mb = x_train[i:i + params['batch_size']], y_train[i:i + params['batch_size']]
                sess.run(model.optimize, feed_dict={model.input: x_train_mb, model.target: y_train_mb, model.is_task1: True, model.is_train: True, model.learning_rate: params['learning_rate']})

            valid_acc = classification_batch_evaluation(sess, model, model.metrics, params['batch_size'], True, x_valid, y=y_valid, stream=True)

            print('valid [{} / {}] valid accuracy: {}'.format(epoch, params['epochs'] + 1, valid_acc))
            logging.info('valid [{} / {}] valid accuracy: {}'.format(epoch, params['epochs'] + 1, valid_acc))

            if valid_acc > initial_best_epoch['valid_acc']:
                initial_best_epoch['epoch'] = epoch
                initial_best_epoch['valid_acc'] = valid_acc
                model.save_model(sess, epoch) 
                ##Saves the model at the following location : 
                ##trained_models/mnist/mnist_10_5/weight_transfer/replication1
                
                ##A
                ##Trying to save model as an HDF5 file
#                 model.save('my_model.h5')


            if epoch - initial_best_epoch['epoch'] >= params['patience']:
                print('Early Stopping Epoch: {}\n'.format(epoch))
                logging.info('Early Stopping Epoch: {}\n'.format(epoch))
                break

        print('Initial training done \n')
        logging.info('Initial training done \n')

        model.restore_model(sess) ##Restores the model after creating it .
        
        #pdb.set_trace()

    transfer_best_epoch = {'epoch': -1, 'train_acc': -1, 'test_acc': -1}
    es_acc = 0.0

    for epoch in range(1, params['epochs'] + 1):
        shuffle = np.random.permutation(len(y_train2))
        x_train2, y_train2 = x_train2[shuffle], y_train2[shuffle]
        for i in range(0, len(y_train2), params['batch_size']):
            x_train_mb, y_train_mb = x_train2[i:i + params['batch_size']], y_train2[i:i + params['batch_size']]
            sess.run(model.optimize, feed_dict={model.input: x_train_mb, model.target: y_train_mb, model.is_task1: False, model.is_train: True, model.learning_rate: params['learning_rate']})

        train_acc = classification_batch_evaluation(sess, model, model.metrics, params['batch_size'], False, x_train2, y=y_train2, stream=True)

        print('train [{} / {}] train accuracy: {}'.format(epoch, params['epochs'] + 1, train_acc))
        logging.info('train [{} / {}] train accuracy: {}'.format(epoch, params['epochs'] + 1, train_acc))

        if train_acc > transfer_best_epoch['train_acc']:
            transfer_best_epoch['epoch'] = epoch
            transfer_best_epoch['train_acc'] = train_acc
            test_acc = classification_batch_evaluation(sess, model, model.metrics, params['batch_size'], False, x_test2, y=y_test2, stream=True)
            transfer_best_epoch['test_acc'] = test_acc

        if epoch % params['patience'] == 0:
            acc_diff = transfer_best_epoch['train_acc'] - es_acc
            if acc_diff < params['percentage_es'] * es_acc:
                print('Early Stopping Epoch: {}\n'.format(epoch))
                logging.info('Early Stopping Epoch: {}\n'.format(epoch))
                break
            es_acc = transfer_best_epoch['train_acc']

    print('Transfer training done \n')
    print('test accuracy: {}'.format(transfer_best_epoch['test_acc']))
    logging.info('Transfer training done \n')
    logging.info('test accuracy: {}'.format(transfer_best_epoch['test_acc']))
    
    

In [ ]:
def get_model(params):
    model, data = None, None
    if params['command'] == 'hist_loss':
        if params['dataset'] == 'mnist':
            model = MNISTHistModel(params)
            data = MNIST(params['data_path']).kntl_data_form(params['t1_train'], params['t1_valid'], 
                params['k'], params['n'], params['t2_test'])
        elif params['dataset'] == 'isolet':
            model = IsoletHistModel(params)
            data = Isolet(params['data_path']).kntl_data_form(250, params['n'], params['k'], params['n'])
        elif params['dataset'] == 'omniglot':
            model = OmniglotHistModel(params)
            data = Omniglot(params['data_path']).kntl_data_form(params['n'], params['k'], params['n'])
        else:
            model = TinyImageNetHistModel(params)
            data = TinyImageNet(params['data_path']).kntl_data_form(350, params['n'], params['k'], params['n'])
    elif params['command'] == 'proto':
        if params['dataset'] == 'mnist':
            model = MNISTProtoModel(params)
            data = MNIST(params['data_path']).kntl_data_form(params['t1_train'], params['t1_valid'], 
                params['k'], params['n'], params['t2_test'])
        elif params['dataset'] == 'isolet':
            model = IsoletProtoModel(params)
            data = Isolet(params['data_path']).kntl_data_form(250, params['n'], params['k'], params['n'])
        elif params['dataset'] == 'omniglot':
            model = OmniglotProtoModel(params)
            data = Omniglot(params['data_path']).kntl_data_form(params['n'], params['k'], params['n'])
        else:
            model = TinyImageNetProtoModel(params)
            data = TinyImageNet(params['data_path']).kntl_data_form(350, params['n'], params['k'], params['n'])
    elif params['command'] == 'weight_transfer':
        if params['dataset'] == 'mnist':
            model = MNISTWeightTransferModel(params)
            data = MNIST(params['data_path']).kntl_data_form(params['t1_train'], params['t1_valid'],params['k'], params['n'], params['t2_test'])
	    
	   
        elif params['dataset'] == 'isolet':
            model = IsoletWeightTransferModel(params)
            data = Isolet(params['data_path']).kntl_data_form(250, params['n'], params['k'], params['n'])
        elif params['dataset'] == 'omniglot':
            model = OmniglotWeightTransferModel(params)
            data = Omniglot(params['data_path']).kntl_data_form(params['n'], params['k'], params['n'])
        else:
            model = TinyImageNetWeightTransferModel(params)
            data = TinyImageNet(params['data_path']).kntl_data_form(350, params['n'], params['k'], params['n'])
	    
    elif params['command'] == 'baseline':
        if params['dataset'] == 'mnist':
            model = MNISTBaselineModel(params)
            data = MNIST(params['data_path']).kntl_data_form(params['t1_train'], params['t1_valid'], 
                params['k'], params['n'], params['t2_test'])
        elif params['dataset'] == 'isolet':
            model = IsoletBaselineModel(params)
            data = Isolet(params['data_path']).kntl_data_form(250, params['n'], params['k'], params['n'])
        elif params['dataset'] == 'omniglot':
            model = OmniglotBaselineModel(params)
            data = Omniglot(params['data_path']).kntl_data_form(params['n'], params['k'], params['n'])
        else:
            model = TinyImageNetBaselineModel(params)
            data = TinyImageNet(params['data_path']).kntl_data_form(350, params['n'], params['k'], params['n'])
    else:
        print('Unknown model type')
        logging.debug('Unknown model type')
        quit()
    #pdb.set_trace()
    return model, data


In [6]:
random.seed(params['random_seed'])
initialization_seq = random.sample(range(50000), params['replications']) 
##initialization_seq = 28883

config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
for rep in range(params['replications']):
    tf.reset_default_graph()  #This does siemthing like clearing the default graph stack and i have no idea what does that mean 
    with tf.Session(config=config) as sess:
            #tf.set_random_seed(initialization_seq[rep])
        np.random.seed(initialization_seq[rep])
            
        model, data = get_model(params)
            #model : weight_transfer_model.MNISTWeightTransferModel
         
        assert not model is None
        assert not data is None
            
        init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
        params['init'] = init
        model.create_saver()
            #self.saver = tf.train.Saver(tf.global_variables(), max_to_keep=1)

        sess.run(init)
            
        rep_path = os.path.join(params['save_dir'], 'replication{}'.format(rep + 1))
        os.mkdir(rep_path)
        model.config['save_dir_by_rep'] = rep_path

        logging.debug('running training/testing')
        #pdb.set_trace()
        if params['command'] == 'baseline':
            train_classification(sess, model, data, params, weight_transfer=False)
        elif params['command'] == 'weight_transfer':
            train_classification(sess, model, data, params, weight_transfer=True)
        elif params['command'] == 'hist_loss':
            train_histogram_loss(sess, model, data, params)
        elif params['command'] == 'proto':
            train_proto_nets(sess, model, data, params)
        else:
            print('Unknown model type')
            logging.debug('Unknown model type')
            quit()
        #pdb.set_trace()
        print("Inside run(params): in run_model.py")


NameError: name 'tf' is not defined

In [ ]:
new_model = keras.models.load_model('model.h5')




In [ ]:
##Creating the t training and testing data set 

In [ ]:
 (x_train, y_train), (x_valid, y_valid), (x_train2, y_train2), (x_test2, y_test2) = data

In [ ]:
import tensorflow as tf
def hello():
    return "Hello world"
def bye():
    return "Good bye"
print(tf.cond(tf.equal(2,2),hello,bye).value)

In [ ]:
with tf.Session() as Sess:
    print(tf.cond(tf.equal(2,2),hello,bye).eval())